In [1]:
%pylab inline
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
import glob
import re
import os
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

work_folder = '/Users/rafa/Dropbox/wind_forecasting_paper/forecasts/EvoDAG2/'

In [3]:
stations = ['aristeomercado',
'cointzio',
'corrales',
'elfresno',
'lapalma',
'lapiedad',
'malpais',
'markazuza',
'melchorocampo',
'patzcuaro']
stations2 = ["20891", "22641", "22887", "23711", "24908", \
            "27947", "28722", "29231", "30230", "37099"]

In [9]:
num_cols = 10
for station in stations:
    files = sorted(glob.glob(work_folder + '*' + station + '*.csv'), key=natural_keys)
    data_osa = None
    data_oda = []
    temp_oda = []
    i = 0
    for fil in files:
        raw = None
        with open(fil, 'rb') as f:
            raw = [float(x.replace('\n', '')) for x in f.readlines()]
        if i == 0:
            data_osa = list(raw)
            temp_oda.append(raw)
            i += 1
        else:
            temp_oda.append(raw)
    temp_oda = np.array(temp_oda)
    for i in range(num_cols):
        data_oda = data_oda + temp_oda[:, i * 24].tolist()
    with open(work_folder + 'OSA/' + station + '.dat', 'wb') as f:
        for d in data_osa:
            f.write(str(d) + '\n')
    with open(work_folder + 'ODA/' + station + '.dat', 'wb') as f:
        for d in data_oda:
            f.write(str(d) + '\n')

In [10]:
num_cols = 10
for station in stations2:
    files = sorted(glob.glob(work_folder + '*' + station + '*.csv'), key=natural_keys)
    data_osa = None
    data_oda = []
    temp_oda = []
    i = 0
    for fil in files:
        raw = None
        with open(fil, 'rb') as f:
            raw = [float(x.replace('\n', '')) for x in f.readlines()]
        if i == 0:
            data_osa = list(raw)
            temp_oda.append(raw)
            i += 1
        else:
            temp_oda.append(raw)
    temp_oda = np.array(temp_oda)
    for i in range(num_cols):
        data_oda = data_oda + temp_oda[:, i * 8].tolist()
    with open(work_folder + 'OSA/' + station + '.dat', 'wb') as f:
        for d in data_osa:
            f.write(str(d) + '\n')
    with open(work_folder + 'ODA/' + station + '.dat', 'wb') as f:
        for d in data_oda:
            f.write(str(d) + '\n')

In [4]:
stations = [u'20891', u'22641', u'22887', u'23711', u'24908', u'27947', u'28722', u'29231', u'30230', u'37099', u'aristeomercado', u'cointzio', u'corrales', u'elfresno', u'lapalma', u'lapiedad', u'malpais', u'markazuza', u'melchorocampo',
            u'patzcuaro']


forecasters = ['NNDE_MSE', 'NNDE_SMAPE', 'EvoDAG2', 'FuzzyForecast', 'ANNGP', 'NN', 'ARIMA', 'NAIVE']
methods = ['OSA', 'ODA']

data = {}
actual_values = {}
i = 0
anex_folder = 'russian stations'
for station in stations:
    data[station] = {}
    for method in methods:
        data[station][method] = {}
    folder = '/Users/rafa/Dropbox/wind_forecasting_paper/datasets/'
    if i > 9:
        anex_folder = 'michoacan stations'
    with open(folder + anex_folder + '/complete/wind_' + station + '_10m_complete.csv', 'rb') as f:
        actual_values[station] = [float(x.split(',')[1].replace('\n', '')) for x in f.readlines()]
    i += 1

In [10]:
len(actual_values['20891'])

102272

In [5]:
for method in methods:
    for forecaster in forecasters:
        if forecaster == 'NAIVE':
            for station in stations:
                num_samples = 240
                if station in stations2:
                    num_samples = 80
                temp = [0] + actual_values[station]
                temp.pop(len(temp) - 1)
                if method == 'OSA':
                    data[station][method][forecaster] = temp[-num_samples:]
                else:
                    temp2 = []
                    for i in range(len(temp) - num_samples, len(temp), 8):
                        temp2 += [temp[i]] * 8
                    data[station][method][forecaster] = temp2
        else:
            folder = '/Users/rafa/Dropbox/wind_forecasting_paper/forecasts/' + forecaster + '/' + method + '/'
            for station in stations:
                forecast = None
                with open(folder + station + '.dat', 'rb') as f:
                    data[station][method][forecaster] = [float(x.replace('\n', '')) for x in f.readlines()]

In [6]:
def smape(a, f):
    """Calculates the symmetric mean average percentage error between two sets a (actual) and f (forecast)"""
    c = list(a)
    d = list(f)
    indexes = []
    for i in range(len(a)):
        if c[i] == 0 and d[i] == 0:
            indexes.append(i)
        if c[i] == -1 or d[i] < 0:
            indexes.append(i)
    c = np.array([x for i, x in enumerate(a) if i not in indexes])
    d = np.array([x for i, x in enumerate(f) if i not in indexes])
    return np.mean(np.abs(d - c) / ((np.abs(c) + np.abs(d)) / 2)) * 100

def mse(a, f):
    """Calculates the Mean Squared Error between two sets a (actual) and f (forecast)"""
    c = list(a)
    d = list(f)
    indexes = []
    for i in range(len(a)):
        if c[i] == 0 and d[i] == 0:
            indexes.append(i)
        if c[i] == -1 or d[i] < 0:
            indexes.append(i)
    c = np.array([x for i, x in enumerate(a) if i not in indexes])
    d = np.array([x for i, x in enumerate(f) if i not in indexes])
    return np.mean(np.power((c - d), 2))

In [7]:
errors_mse = {}
errors_smape = {}

for method in methods:
    errors_mse[method] = {}
    errors_smape[method] = {}
    for station in stations:
        errors_mse[method][station] = {}
        errors_smape[method][station] = {}
        for forecaster in forecasters:
            num_samples = 240
            if station in stations2:
                num_samples = 80
            try:
                errors_mse[method][station][forecaster] = mse(actual_values[station][-num_samples:], data[station][method][forecaster])
                errors_smape[method][station][forecaster] = smape(actual_values[station][-num_samples:], data[station][method][forecaster])
            except ValueError:
                print method + ' ' + forecaster + ' ' + station

/Library/Python/2.7/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Python/2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [12]:
print round(mse(actual_values['lapiedad'][-num_samples:], np.zeros(240)), 4)
print smape(actual_values['lapiedad'][-num_samples:], np.zeros(240))

52.6762
200.0


In [98]:
def best(errors):
    bdict = {}
    for method in methods:
        bdict[method] = {}
        for station in stations:
            bdict[method][station] = [] 
            best_value = 9999999999
            best_name = ''
            for forecaster in forecasters:
                if round(errors[method][station][forecaster], 4) < best_value:
                    best_value = round(errors[method][station][forecaster], 4)
                    best_name = forecaster
            bdict[method][station].append(best_name)
    return bdict

In [99]:
def replace_last(source_string, replace_what, replace_with):
    head, _sep, tail = source_string.rpartition(replace_what)
    return head + replace_with + tail


def print_table(errors, metric, bdict):
    forecasters_print = list(forecasters)
    if metric == 0:
        forecasters_print.pop(1)
    else:
        forecasters_print.pop(0)
    for method in methods:
        if method == 'ODA':
            forecasters_print.pop(forecasters_print.index('NAIVE'))
        print method
        row1 = '                              '
        for forecaster in forecasters_print:
            spaces = ' '
            # for i in range(0, 15 - len(forecaster)):
            #     spaces += ' '
            row1 += spaces + forecaster + spaces + '&'
        print replace_last(row1, '&', '\\\\')
        print '\hline'
        for station in stations:
            row2 = ''
            for forecaster in forecasters_print:
                    result = str(round(errors[method][station][forecaster], 4))
                    spaces = ' '
                    # for i in range(0, 15 - len(result)):
                    #     spaces += ' '
                    best = ''
                    if bdict[method][station][0] == forecaster:
                        best = '\\bfseries'
                    row2 += spaces + best + result + spaces + '&'
            for i in range(0, 15 - len(station)):
                station += ' '
            print replace_last(station + ' & ' + row2, '&', '\\\\')
        print '\hline'

In [100]:
bdict_mse = best(errors_mse)
bdict_smape = best(errors_smape)
print_table(errors_mse, 0, bdict_mse)
print_table(errors_smape, 1, bdict_smape)

OSA
                               NNDE_MSE & EvoDAG2 & FuzzyForecast & ANNGP & NN & ARIMA & NAIVE \\
\hline
20891           &  \bfseries1.1446 & 1.2099 & 1.754 & 14.1139 & 1.3324 & 1.1478 & 1.4805 \\
22641           &  \bfseries1.2404 & 1.5381 & 2.1581 & 3.8497 & 1.4735 & 1.3916 & 2.0972 \\
22887           &  0.9639 & 0.907 & 1.1022 & 1.9551 & 1.3024 & \bfseries0.8688 & 1.1692 \\
23711           &  0.2887 & 0.3358 & 0.7066 & 0.3689 & 0.3743 & \bfseries0.2822 & 0.5091 \\
24908           &  \bfseries0.292 & 0.3392 & 3.8116 & 0.4593 & 0.3572 & 0.3951 & 0.9189 \\
27947           &  \bfseries1.7919 & 2.0307 & 2.3074 & 11.8573 & 1.9456 & 2.0578 & 2.5132 \\
28722           &  \bfseries1.8658 & 2.1941 & 3.1781 & 9.0612 & 2.0727 & 1.9485 & 2.8462 \\
29231           &  \bfseries0.6496 & 0.7641 & 1.1799 & 3.1808 & 0.7296 & 0.7193 & 1.0132 \\
30230           &  \bfseries0.2412 & 0.3486 & 4.7282 & 0.2871 & 0.3043 & 0.3214 & 0.6731 \\
37099           &  \bfseries0.5315 & 0.5652 & 0.959 & 2.2647 & 0